In [ ]:
""" Imports and cleans the raw data """
""" saves the cleaned data for each month in the data_YYYYMM variable"""

" Import needed packages"
import pandas as pd
import requests
from zipfile import ZipFile
from io import BytesIO


""" Select the data to elaborate and some options """ 

" List the year and month of the data that you want to import as YYYYMM "
dates = ["202204", "202205","202206","202207","202208","202209"
         ,"202210","202211","202212","202301","202302","202303"]

" Import data from the web repository or from a local folder "
import_from_local = False # True if you want to import from local.
                         # False (default) to import from web repository.
folder_path = "" # For local import, path to the folder where data 
                # is stored. Default is the working folder as "".
    
" Do you want to save the cleaned data locally? "
save_cleaned_data = True # False (default) to not save the data
                        # True to save the data locally
clean_folder = "" # path to the folder where the cleaned data will be
                # saved. Default is the working folder as "".

    
""" The computation part of the script starts from here """

for date in dates: # iterates over all the selected files
    
    # Header
    print(date + " cleaning in progress ################################")
    
    # Imports the data from web as Pandas DataFrames
    if import_from_local is False:
        print("Importing data from the web")
        url_repository = "https://divvy-tripdata.s3.amazonaws.com/"
        trail_file_name = "-divvy-tripdata.zip"
        url = url_repository + date + trail_file_name
        response = requests.get(url)
        zipfile = ZipFile(BytesIO(response.content))
        csvfile = zipfile.open(zipfile.namelist()[0])
        data = pd.read_csv(csvfile)
    
    # Imports the cvs files from the local working folder 
    # as Pandas DataFrames
    if import_from_local is True:
        path = folder_path + date + "-divvy-tripdata.csv"
        data = pd.read_csv(path)
    
    # Displays some info of the imported data
    print(data.head(), "\n") # displays the first rows
    data.info()

    # Eliminates the empty rows
    print("\nEliminating empty rows")
    data.dropna(inplace=True)

    # Eliminates the duplicate rows
    print("Eliminating duplicate rows")
    data.drop_duplicates(inplace=True)

    # Turns the entries of the columns "started_at" and "ended_at" 
    # into datetime format 
    print("""Turning the entries of the columns "started_at" 
    and "ended_at" into datetime format""")
    data["started_at"] = pd.to_datetime(data["started_at"])
    data["ended_at"] = pd.to_datetime(data["ended_at"])


    # Shows all the values of the columns "rideable_type", 
    # "member_casual", so I can check that they are valid. 
    print("""values of the columns "rideable_type", "member_casual" """)
    for i in ["rideable_type", "member_casual"]: 
        data_member = data.groupby(i)
        entries = data_member.groups.keys()
        print(i, " : ",entries)
        print(" ")

    # Shows the max and min of all the data, 
    # so I can check that the boundaries of the data make sense.
    print("Shows the MAX and MIN of all the data")
    data_max = pd.DataFrame(
                            [data.max(), 
                             data.min()]
                            , index = ["Max", "Min"]).transpose()
    print(data_max)

    # Add the column "ride_length" 
    # as subtraction between "ended_at" and "started_at"
    print("\nAdding the column ride_length")
    data["ride_length"] = data["ended_at"] - data["started_at"]

    # Add the column "day_of_week" that shows the day of the week 
    # with 0 as Monday and 6 as Sunday 
    print("Adding the column day_of_week")
    data["day_of_week"] = data["started_at"].dt.dayofweek

    # Checks if there are negative or zero ride_length values 
    # and removes these rows
    print("Removing the rows with negative ride length ")
    zero_time = pd.Timedelta("0 days 00:00:00")
    right_data = data[data.ride_length > zero_time]
    data = right_data

    # Shows max and min for the new columns,
    # so I can check their boundaries
    print("Shows max and min for the new columns")
    print(pd.DataFrame(
        [data[["ride_length","day_of_week"]].max(),
         data[["ride_length","day_of_week"]].min()]
        , index=["Max", "Min"]).transpose())

    # Saves the cleaned data in a csv file
    if save_cleaned_data is True:
        file_name = clean_folder + "cleaned_" + date + ".csv"
        data.to_csv(file_name)
        print(f"\nSaving the cleaned data in the {file_name} file")
    
    # Saves the cleaned data in a global variable for further use 
    data_name = "data_" + date
    globals()[data_name] = data
    print(f"\nSaving the cleaned data in the {data_name} variable")
    
    # If running in Jupiter environment, tries to store the data
    # for use in a different kernel
    try:
        if get_ipython():
            # Running in a Jupyter environment?
            %store globals()[data_name]
            print("Storing data for use in a different kernel")
    except:
        print("Not running in a Jupyter environment, may not save global variables")
        pass
    
    # Shows some info about the cleaned data
    print("\nInfos of the cleaned data ")
    print(data.info())